# Prepare H2O and data

In [3]:
import h2o
h2o.init(max_mem_size = "300G")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_352"; OpenJDK Runtime Environment (build 1.8.0_352-8u352-ga-1~20.04-b08); OpenJDK 64-Bit Server VM (build 25.352-b08, mixed mode)
  Starting server from /usr/local/lib/python3.8/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp7zl_24wi
  JVM stdout: /tmp/tmp7zl_24wi/h2o_root_started_from_python.out
  JVM stderr: /tmp/tmp7zl_24wi/h2o_root_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,1 month and 8 days
H2O_cluster_name:,H2O_from_python_root_6ejtau
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,266.6 Gb
H2O_cluster_total_cores:,96
H2O_cluster_allowed_cores:,96
H2O_cluster_status:,"locked, healthy"


In [4]:
import pandas as pd
from utils import Timer

data_path = "./"

with Timer(f"Load train full"):
    train_data = pd.read_parquet(f"{data_path}/featuretools_humanknowledge_nyc_taxi_55M.parquet")

with Timer("split train and test"):
    test_sample = train_data.sample(frac = 0.1)
    train_sample = train_data.drop(test_sample.index)

with Timer("convert to h2o frame"):
    h2o_train_data = h2o.H2OFrame(train_sample)

Load train full took 13.972860928624868 sec
split train and test took 21.15672191325575 sec
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
convert to h2o frame took 1033.736319795251 sec


# Run H2O autoML

In [5]:
from h2o.automl import H2OAutoML

x = h2o_train_data.columns
y = "fare_amount"

aml = H2OAutoML()
with Timer("h2o automl train"):
    aml.train(y=y, training_frame=h2o_train_data)

AutoML progress: |█████████████████Failed polling AutoML progress log: Local server has died unexpectedly. RIP.
Job request failed Local server has died unexpectedly. RIP., will retry after 3s.
Job request failed Local server has died unexpectedly. RIP., will retry after 3s.
Job request failed Local server has died unexpectedly. RIP., will retry after 3s.
Job request failed Local server has died unexpectedly. RIP., will retry after 3s.
Job request failed Local server has died unexpectedly. RIP., will retry after 3s.
Job request failed Local server has died unexpectedly. RIP., will retry after 3s.
Job request failed Local server has died unexpectedly. RIP., will retry after 3s.
Job request failed Local server has died unexpectedly. RIP., will retry after 3s.
Job request failed Local server has died unexpectedly. RIP., will retry after 3s.
Job request failed Local server has died unexpectedly. RIP., will retry after 3s.
Job request failed Local server has died unexpectedly. RIP., will re

H2OConnectionError: Local server has died unexpectedly. RIP.

In [ ]:
with Timer("convert test to h2o frame"):
    h2o_test_data = h2o.H2OFrame(test_sample)

with Timer("predict test with h2o model"):
    preds = aml.predict(h2o_test_data)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

y_val = test_sample["fare_amount"].to_numpy()
pred_val = preds.as_data_frame()['predict'].to_numpy()
rmse = np.sqrt(mean_squared_error(y_val, pred_val))
print('LightGBM RMSE', rmse)

In [ ]:
 with Timer("explain h2o_test_data"):
    aml.explain(h2o_test_data)

# Backup

In [5]:
data.summary()

/tmp/ipykernel_162380/605889900.py:1: H2ODeprecationWarning: ``summary()`` is deprecated, please use ``show_summary()`` or ``get_summary()`` instead
  data.summary()


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
type,time,real,string,real,real,real,real,int
mins,1230768027000.0,-300.0,NaN,-3442.059565,-3492.263768,-3442.024565,-3547.886698,0.0
mean,1332289129821.6658,11.345045601662932,NaN,-72.509684443588,39.919791786888425,-72.51120972971923,39.92068144482933,1.6853799201556823
maxs,1435708794000.0,93963.36,NaN,3457.625683,3408.789565,3457.62235,3537.132528,208.0
sigma,58317921008.69085,20.71083219823222,NaN,12.848883381402695,9.642353041994898,12.782196517830833,9.633345796415139,1.3276643570959694
zeros,0,1380,0,1055693,1052158,1052745,1049666,195416
missing,0,0,0,0,0,376,376,0
0,2009-06-15 17:26:21,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.84161,40.712278,1.0
1,2010-01-05 16:52:16,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1.0
2,2011-08-18 00:35:00,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.76127,-73.991242,40.750562,2.0
